In [ ]:
# Importar las bibliotecas necesarias para interactuar con la API de OpenAI y manejar variables de entorno
import openai
import os
import elevenlabs
from elevenlabs import play
from elevenlabs.client import ElevenLabs

# Configurar la clave de la API de OpenAI utilizando una variable de entorno para mayor seguridad
openai.api_key = "Openai_ API_Key"

# Definir la función que utiliza la API de OpenAI para generar historias basadas en un contexto y un prompt específicos
def generar_historia(context, prompt):
    # Preparar ejemplos para el few-shot prompting, estructurando el diálogo para guiar a la IA
    few_shot_examples = [
        {"role": "system", "content": "Contexto: " + context},
        {"role": "user", "content": "Usuario: " + prompt},
        {"role": "system", "content": "Sistema: ¿Puedes proporcionar más detalles sobre la historia?"},
        {"role": "user", "content": "Usuario: " + prompt + "\nMás detalles sobre la historia."},
    ]
    
    # Realizar la petición a la API para generar la historia, especificando el modelo y los parámetros de la solicitud
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=few_shot_examples,
        temperature=1.2,
        max_tokens=400
    )
    # Extraer el mensaje generado por la IA de la respuesta
    message = response.choices[0]['message']
    # Devolver el contenido del mensaje generado
    return message['content']

# Solicitar al usuario los inputs necesarios para generar la historia: contexto, personajes y detalles adicionales
contexto = input("Escribe el contexto general de la historia:")
personajes = input("Describe tu personaje principal y secundarios:")
detalles_adicionales = input("Proporciona detalles adicionales sobre el entorno, conflictos, objetivos, etc. (opcional):")

# Validar que se hayan proporcionado los inputs mínimos requeridos para generar la historia
if not contexto.strip() or not personajes.strip():
    print("Debes proporcionar un contexto general de la historia y describir al menos un personaje principal.")
    exit()

# Construir el prompt combinando los inputs del usuario, ajustándose a la necesidad de los detalles adicionales
prompt = f"{personajes}\n{detalles_adicionales}" if detalles_adicionales else personajes

# Definir un prompt adicional para optimizar la historia, solicitando mejoras y detalles para enriquecer la narrativa
# Prompt de optimización= mejora y detalla la siguiente historia, agregando elementos sorprendentes, giros inesperados y descripciones vívidas para cautivar al lector y sumergirlo en la narrativa:
prompt_optimizacion = """mejora y detalla la siguiente historia, agregando elementos sorprendentes, giros inesperados y descripciones vívidas para cautivar al lector y sumergirlo en la narrativa:"""

# Generar la historia optimizada utilizando la función definida anteriormente y los prompts del usuario
historia_optimizada = generar_historia(contexto, prompt_optimizacion)

# Imprimir la historia optimizada
print("Historia Generada (Optimizada):")
print(historia_optimizada)

# Preparar dos prompts a partir de la historia optimizada para generar imágenes, dividiendo la historia en dos partes
prompts_imagenes = [
    historia_optimizada[:len(historia_optimizada)//2],  # Primera mitad de la historia
    historia_optimizada[len(historia_optimizada)//2:],  # Segunda mitad de la historia
]

# Inicializar la lista para almacenar las URLs de las imágenes generadas
def generar_imagenes(texto):
    urls_imagenes = []
    # Dividir el texto en segmentos de 1000 caracteres máximo
    segmentos = [texto[i:i+1000] for i in range(0, len(texto), 1000)]
    
    for segmento in segmentos:
        try:
            response = openai.Image.create(
                prompt=segmento,
                n=1
            )
            # Asumiendo que la respuesta incluye al menos una imagen
            url_imagen = response['data'][0]['url']
            urls_imagenes.append(url_imagen)
        except Exception as e:
            print(f"Error al generar imagen para el segmento: {e}")
    
    return urls_imagenes

# Ejemplo de uso y otras partes del código omitidas por brevedad...

# Generar y mostrar URLs de imágenes después de la generación de la historia optimizada
urls_imagenes = generar_imagenes(historia_optimizada)
print("URLs de las imágenes generadas:")
for url in urls_imagenes:
    print(url)


# integrar la generación de audio (Requiere implementación adicional)
client = ElevenLabs(
api_key = "ElevenLabs API-Key",
)

voice1 = elevenlabs.Voice(
    voice_id = "z9fAnlkpzviPz146aGWa", #Voice of Glinda
    settings = elevenlabs.VoiceSettings(
        stability = 0.7,
        similarity_boost = 0.75,
        style=0.0,
        use_speaker_boost=True
    )
)

audio = client.generate(
    text = historia_optimizada,
    voice = voice1,
    model = "eleven_multilingual_v2"
)

elevenlabs.play(audio)
elevenlabs.save(audio, "audio.mp3")



# (Opcional) Integración para generación de contenido de video
def generar_video(historia, urls_imagenes):
    # Esta función podría combinar las imágenes generadas y el audio en un video.
    # La implementación específica dependerá de las herramientas y APIs disponibles.
    pass

